In [37]:
import model.ChainCRF
import importlib
import dataLoad.lload
importlib.reload(model.ChainCRF)
importlib.reload(dataLoad.lload)
from  dataLoad.lload import CustomDataset
from  dataLoad.lload import *

from model.ChainCRF import BLITM
import torchtext
from get_data import *
from check import *
from collections import Counter,defaultdict
embed_size = 100
language = "Chinese"
min_freq = 2

In [38]:

train_data = get_train_data(language)
import os
import torchtext
train_word = [word  for sentence in train_data for word,label in sentence]
print(len(train_word))
vocab = torchtext.vocab.vocab(Counter(train_word),min_freq=min_freq,specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])
print(vocab.get_default_index())
torch.save(vocab.state_dict(),"./bilstm_crf/pretrain/{}.pt".format(language))
def sent2word(sentence):
    return [w for w, _ in sentence]
def sent2label(sentence):
    return [l for _, l in sentence]
max_length = max([len(l) for l in train_data])
max_length = max(max_length, 256)
sorted_labels = sorted_labels_chn if language == 'Chinese' else sorted_labels_eng
def label2index(label):
    return sorted_labels.index(label)

123967
0


In [39]:

# Create a custom dataset
custom_dataset = CustomDataset(train_data, vocab, label2index, max_length)
# Create a DataLoader
batch_size = 32
dataloader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)

custom_dataset[0]

{'word_embeddings': tensor([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
         19,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         

In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import numpy as np
def train(model, train_loader, num_epochs, learning_rate, device):
    # Move the model to GPU
    model.to(device)
    criterion = nn.CrossEntropyLoss(ignore_index=-1)  # ignore padding index

    # Define the optimizer
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    # Create a progress bar
    progress_bar = tqdm(total=num_epochs * len(train_loader))

    for epoch in range(num_epochs):
        model.train()
        sum_loss = 0

        for batch in train_loader:
            # Move inputs, labels, and mask to GPU
            input_ = batch['word_embeddings']
            label_ = batch['label_indices']
            mask_ = batch['mask'].to(device)
            length = batch['max_length']
            max_length = np.argmax(length)
            aaa = length[max_length]
            inputs =torch.Tensor(input_[:,:aaa])
            labels =torch.Tensor( label_[:,:aaa])
            mask = torch.Tensor(mask_[:,:aaa])
            inputs = inputs.to(device)
            labels = labels.to(device)
            mask = mask.to(device)

            # Zero the gradients
            optimizer.zero_grad()


            # Forward pass
            # loss = model(input_, mask_, label_)
            outputs = model(inputs)
            xx = outputs.shape[-1]
            outputs = outputs.view(-1,xx)
            labels = labels.reshape(-1)

            # Compute the loss
            loss = criterion(outputs, labels)

            # Mask out the padding positions
            # loss = (loss * mask.reshape(-1)).sum() / mask.sum()

            # Backward pass
            loss.backward()
            sum_loss += loss.item()

            # Update the parameters
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
            optimizer.step()
            progress_bar.update(1)

        progress_bar.set_postfix_str("Epoch:{}, Loss:{}".format(epoch + 1, sum_loss / len(train_loader)))

    # Close the progress bar
    progress_bar.close()

# Example usage:
# Assuming you have a model, train_loader, num_epochs, and learning_rate



In [41]:
hidden_dim = 100
bilstm = BLITM(len(sorted_labels),len(vocab),embed_size,hidden_dim)
file = "./bilstm_crf/pretrain/BILSTM_{}.bin".format(language)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

mode = True
if mode:
    train(bilstm,dataloader,10,1e-2,device)
    torch.save(bilstm.state_dict(),file)
else:
    bilstm.load_state_dict(torch.load(file))

100%|██████████| 1200/1200 [01:16<00:00, 15.62it/s, Epoch:10, Loss:0.06934428222787876]


In [42]:
def mycheck(language, vocab, res_file, model, train_or_valid,device):
    valid = get_data_from_file(res_file)
    pred_path = "example_data/BILSTM_{}_{}.txt".format(language, "train" if train_or_valid else "dev")
    valid_data = CustomDataset(valid, vocab, label2index, 250)
    valdataloader = DataLoader(valid_data, batch_size=64, shuffle=False)

    # Move the model to GPU
    # device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    with open(pred_path, "w") as f:
        with torch.no_grad():
            iter = 0
            for batch in valdataloader:
                max_length = np.argmax(batch['max_length'])
                aaa = batch['max_length'][max_length]
                word_embeddings = batch['word_embeddings'][:,:aaa]

                preds = model(word_embeddings)
                _,preds= torch.max(preds,dim=-1)

                for pred in preds:
                    pred_labels = []
                    for i in range(len(valid[iter])):
                        f.write(valid[iter][i][0] + " " + sorted_labels[pred[i]] + '\n')
                        pred_labels.append(sorted_labels[pred[i]])
                    f.write('\n')
                    iter = iter + 1

    # Move the model back to CPU if needed
    model.to("cpu")

    check(language, "{}/{}.txt".format(language, "train" if train_or_valid else "validation"), pred_path)

# Example usage:
# Assuming you have a model, vocab, res_file, max_length, train_or_valid, label2index, sorted_labels
# Make sure to replace the imports with the actual modules and functions in your code


In [43]:
device = torch.device("cpu")

# mycheck(language=language,vocab=vocab,res_file="{}/train.txt".format(language),model=bilstm_crf,train_or_valid=1,device=device)
mycheck(language=language,vocab=vocab,res_file="{}/validation.txt".format(language),model=bilstm,train_or_valid=0,device=device)
# loss:114 0.3803  0.3954
# -92.54755198711143 0.3460 0.3643

              precision    recall  f1-score   support

      B-NAME     0.9400    0.9216    0.9307       102
      M-NAME     0.8072    0.8933    0.8481        75
      E-NAME     0.9798    0.9510    0.9652       102
      S-NAME     0.7778    0.8750    0.8235         8
      B-CONT     1.0000    0.9394    0.9688        33
      M-CONT     1.0000    0.9375    0.9677        64
      E-CONT     1.0000    1.0000    1.0000        33
      S-CONT     0.0000    0.0000    0.0000         0
       B-EDU     0.9533    0.9623    0.9577       106
       M-EDU     0.9778    0.9944    0.9860       177
       E-EDU     0.9626    0.9717    0.9671       106
       S-EDU     0.0000    0.0000    0.0000         0
     B-TITLE     0.9020    0.9086    0.9053       689
     M-TITLE     0.8815    0.9304    0.9053      1479
     E-TITLE     0.9897    0.9797    0.9847       689
     S-TITLE     0.0000    0.0000    0.0000         0
       B-ORG     0.9551    0.8966    0.9249       522
       M-ORG     0.9571    